In [218]:
import sys
import pandas as pd
import numpy as np
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

# 배민

In [219]:
baemin_orders = pd.read_csv(r"E:\d_down\baemin_orders_[음식배달]_닭도리탕_전문_도리당_가락점_unknown_20251230.csv", encoding='utf-8')
baemin_orders['platform'] = '배민'

baemin_orders = baemin_orders.drop(columns=['결제타입', '결제금액', '즉시할인_배민지원',
                                            '배민부담_쿠폰할인', '주문중개', '고객할인비용'
                                            ])

baemin_orders.rename(columns={'주문상태':'order_status',
                              '주문번호' : 'order_id',
                              '주문시각' : 'order_date',
                              '광고상품' : 'ad_product',
                              '캠페인ID': 'ad_id',
                              '주문내역' : 'order_summary',
                              '주문수량' : 'option_qty',
                              '수령방법' : 'delivery_type',
                              '총결제금액' : 'total_amount',
                              '상품금액' : 'menu_amount',
                              '즉시할인' : 'instant_discount_coupon',
                              '부가세' : 'fee_vat',
                              '정산금액' : 'settlement_amount',
                              '만나서결제금액' : 'fee_meet_pay_amount',
                              '그외' : 'fee_etc',
                              '입금예정금액' : 'settlement_amount',
                              '배달' : 'fee_delivery',
                              '주문옵션금액' : 'menu_option_price',
                              '주문옵션상세' :'menu_option_name',
                              "즉시할인_파트너부담" : "fee_ad"
                              
                              }, inplace=True)

baemin_orders['store_id'] = np.nan

baemin_orders['instant_discount_coupon_YN'] = np.where(baemin_orders.groupby('order_id')["instant_discount_coupon"].transform('sum') > 0, 'Y', 'N')

baemin_orders = baemin_orders[['platform','order_date','store_id','store_name',
       'ad_id', 'ad_product', 'order_id','order_summary','menu_option_name', 'option_qty', 'delivery_type',
       'total_amount', 'menu_amount', 'menu_option_price', 'instant_discount_coupon', 'instant_discount_coupon_YN',
       'fee_ad','settlement_amount','collected_at']]

baemin_orders['menu_option_price'] = np.nan
baemin_orders = baemin_orders.rename(columns={'instant_discount_amount':'instant_discount_coupon'})

# date 변환
baemin_orders['order_date'] = baemin_orders['order_date'].str.replace(r'\([월화수목금토일]\)', '', regex=True)
baemin_orders['order_date'] = baemin_orders['order_date'].str.replace('오전', 'AM').str.replace('오후', 'PM')
baemin_orders['order_date'] = pd.to_datetime(baemin_orders['order_date'], format='%Y. %m. %d.  %p %I:%M:%S', errors='coerce')
baemin_orders['collected_at'] = pd.to_datetime(baemin_orders['collected_at'])

# 쿠팡


In [ ]:
coupang_orders = pd.read_csv(r"E:\d_down\coupangeats_orders_닭도리탕_전문_도리당_가락점_814147_20251230.csv", encoding='utf-8')
coupang_df = coupang_orders.copy()

cond_df = coupang_df[coupang_df["is_cancelled"] == 'N']
cond = cond_df['order_id'].unique()
coupang_df = coupang_df[coupang_df['order_id'].isin(cond)]
coupang_df['platform'] = '쿠팡'


coupang_df['ad_id'] = np.nan
coupang_df['ad_product'] = np.nan

coupang_dfs = coupang_df [['platform', 'order_date', 'store_id', 'store_name', 'ad_id', 'ad_product',
 'order_id','order_summary', 'menu_name', 'menu_qty','delivery_type',
    'menu_price', 
       '매출액', '광고비','상점부담_쿠폰',
       '정산_예정_금액', 'collected_at']]

coupang_dfs.rename(columns={'menu_name':'menu_option_name',
                             'menu_qty':'option_qty',
                             'menu_price':'total_amount',
                             'menu_options':'menu_option_price',
                             '매출액': 'menu_price',
                             '광고비' : 'fee_ad',
                             '상점부담_쿠폰' : 'instant_discount_coupon',
                             '정산_예정_금액' : 'settlement_amount'
                             }, inplace=True)

coupang_dfs = coupang_dfs[
       ['platform', 'order_date', 'store_id', 'store_name', 'ad_id',
       'ad_product', 'order_id', 'order_summary', 'menu_option_name',
       'option_qty', 'delivery_type', 'total_amount', 'fee_ad',
       'instant_discount_coupon', 'settlement_amount', 'collected_at']]

coupang_dfs['menu_amount'] = coupang_dfs['total_amount']
coupang_dfs['menu_option_price'] = np.nan
coupang_dfs['instant_discount_amount_YN'] = np.where(coupang_dfs.groupby('order_id')["instant_discount_coupon"].transform('sum') > 0, 'Y', 'N')

coupang_dfs = coupang_dfs[['platform', 'order_date', 'store_id', 'store_name', 'ad_id',
       'ad_product', 'order_id', 'order_summary', 'menu_option_name',
       'option_qty', 'delivery_type', 'total_amount', 'menu_amount',
       'menu_option_price', 'instant_discount_coupon',
       'instant_discount_amount_YN', 'fee_ad', 'settlement_amount',
       'collected_at']]

coupang_dfs["order_date"] = pd.to_datetime(coupang_dfs["order_date"])
coupang_dfs["collected_at"] = pd.to_datetime(coupang_dfs["collected_at"])
coupang_df = coupang_dfs

C:\Users\민준\AppData\Local\Temp\ipykernel_2788\2238527492.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupang_dfs.rename(columns={'menu_name':'menu_option_name',


# 병합

# 병합 테이블

In [222]:
orders_concat = pd.concat([baemin_orders, coupang_dfs], axis=0).reset_index(drop=True)

In [223]:
orders_concat.head()

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,option_qty,delivery_type,total_amount,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,fee_ad,settlement_amount,collected_at,instant_discount_amount_YN
0,배민,2025-12-28 22:01:46,NaN,[음식배달] 닭도리탕 전문 도리당 가락점,12597302.0,배민1플러스 (배민클럽),T2920001QRX0,[한우 대창] 한우 순살 곱도리탕,[한우 대창] 한우 순살 곱도리탕,1,한집배달,26900.0,27900.0,NaN,3000.0,Y,2000.0,19296.0,2025-12-30 01:28:44.977000+00:00,NaN
1,배민,2025-12-28 22:01:46,NaN,[음식배달] 닭도리탕 전문 도리당 가락점,12597302.0,배민1플러스 (배민클럽),T2920001QRX0,[한우 대창] 한우 순살 곱도리탕,[후.참] 계란,1,한집배달,NaN,NaN,NaN,NaN,Y,NaN,NaN,2025-12-30 01:28:44.977000+00:00,NaN
2,배민,2025-12-28 22:01:46,NaN,[음식배달] 닭도리탕 전문 도리당 가락점,12597302.0,배민1플러스 (배민클럽),T2920001QRX0,[한우 대창] 한우 순살 곱도리탕,기본 치킨무 빼주세요,1,한집배달,NaN,NaN,NaN,NaN,Y,NaN,NaN,2025-12-30 01:28:44.977000+00:00,NaN
3,배민,2025-12-28 22:01:46,NaN,[음식배달] 닭도리탕 전문 도리당 가락점,12597302.0,배민1플러스 (배민클럽),T2920001QRX0,[한우 대창] 한우 순살 곱도리탕,대파 추가,1,한집배달,NaN,NaN,NaN,NaN,Y,NaN,NaN,2025-12-30 01:28:44.977000+00:00,NaN
4,배민,2025-12-28 22:01:46,NaN,[음식배달] 닭도리탕 전문 도리당 가락점,12597302.0,배민1플러스 (배민클럽),T2920001QRX0,[한우 대창] 한우 순살 곱도리탕,중간매운맛(신라면 보다 좀 더 매움),1,한집배달,NaN,NaN,NaN,NaN,Y,NaN,NaN,2025-12-30 01:28:44.977000+00:00,NaN


# 이상치 설정

In [224]:
sales_employee = pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\sales_employee.csv")
sales_employee

,오픈순서,호점,매장명,사업자 번호,점주명,담당자,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at,email,쿠팡_매장명,배민_매장명,collection_date,composite_key
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,203-30-08476_배달의 민족_2026-01-02
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,203-30-08476_요기요_2026-01-02
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,203-30-08476_쿠팡이츠_2026-01-02
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,203-30-08476_땡겨요_2026-01-02
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,203-30-08476_토더_2026-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,쿠팡이츠,ss813847,7642shin@@,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,229-229-01649_쿠팡이츠_2026-01-02
344,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,땡겨요,ss813847,7642shin@@,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,229-229-01649_땡겨요_2026-01-02
345,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,배달의 민족,a4140172506,b4140172506,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,414-01-72506_배달의 민족_2026-01-02
346,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,요기요,a4140172506,b$!40172506,2026-01-02 04:19:23,NaN,NaN,NaN,2026-01-02,414-01-72506_요기요_2026-01-02


In [225]:

# 이름 정규화(공백 제거 등) — 선택
orders = orders_concat.copy()
sales  = sales_employee.copy()

orders['store_name_norm'] = orders['store_name'].astype(str).str.strip()
sales['쿠팡_매장명_norm'] = sales['쿠팡_매장명'].astype(str).str.strip()
sales['배민_매장명_norm'] = sales['배민_매장명'].astype(str).str.strip()

# 쿠팡 기준 LEFT JOIN
left_coupang = orders.merge(
    sales, how='left',
    left_on='store_name_norm', right_on='쿠팡_매장명_norm',
    suffixes=('', '_se')
)
left_coupang['플랫폼'] = '쿠팡이츠'

# 배민 기준 LEFT JOIN
left_baemin = orders.merge(
    sales, how='left',
    left_on='store_name_norm', right_on='배민_매장명_norm',
    suffixes=('', '_se')
)
left_baemin['플랫폼'] = '배달의민족'

# 두 결과를 한데 모으기 (플랫폼별 1행씩 생김)
result = pd.concat([left_coupang, left_baemin], ignore_index=True)

# 필요 없는 보조 키 제거
result = result.drop(columns=['store_name_norm', '쿠팡_매장명_norm', '배민_매장명_norm'], errors='ignore')

# 확인
orders_concat = result.copy()


In [226]:
orders_concat.columns


Index(['platform', 'order_date', 'store_id', 'store_name', 'ad_id',
       'ad_product', 'order_id', 'order_summary', 'menu_option_name',
       'option_qty', 'delivery_type', 'total_amount', 'menu_amount',
       'menu_option_price', 'instant_discount_coupon',
       'instant_discount_coupon_YN', 'fee_ad', 'settlement_amount',
       'collected_at', 'instant_discount_amount_YN', '오픈순서', '호점', '매장명',
       '사업자 번호', '점주명', '담당자', '실오픈일', '상세주소', '광역', '시군구', '읍면동', '플랫폼',
       '계정ID', '계정PW', 'collected_at_se', 'email', '쿠팡_매장명', '배민_매장명',
       'collection_date', 'composite_key'],
      dtype='object')

In [227]:
import datetime as dt

In [228]:
orders_concat["order_daily"] = orders_concat["order_date"].dt.date

In [229]:
orders_grp_daily = orders_concat.groupby(["order_daily", "매장명", "담당자", "email", "광역", "시군구", "읍면동"]).agg(
    total_order_count=('order_id', 'nunique'),
    total_amount=('total_amount', 'sum')
).reset_index()

In [230]:
orders_grp_daily.head()

,order_daily,매장명,담당자,email,광역,시군구,읍면동,total_order_count,total_amount
0,2025-11-01,도리당 가락점,김대진 팀장,a17019@kakao.com,서울,송파구,송이로17길,8,974500.0
1,2025-11-02,도리당 가락점,김대진 팀장,a17019@kakao.com,서울,송파구,송이로17길,21,2371500.0
2,2025-11-04,도리당 가락점,김대진 팀장,a17019@kakao.com,서울,송파구,송이로17길,12,1286500.0
3,2025-11-05,도리당 가락점,김대진 팀장,a17019@kakao.com,서울,송파구,송이로17길,8,856000.0
4,2025-11-06,도리당 가락점,김대진 팀장,a17019@kakao.com,서울,송파구,송이로17길,5,518000.0


In [231]:
orders_grp_daily["ARPU"] = orders_grp_daily["total_amount"] / orders_grp_daily["total_order_count"]
orders_grp_daily = orders_grp_daily[['order_daily', '담당자', "email", '매장명','광역', '시군구', '읍면동', 'total_order_count',
       'total_amount', 'ARPU']]

In [232]:
orders_grp_daily["order_daily"] = pd.to_datetime(orders_grp_daily["order_daily"], format='%Y-%m-%d')
orders_grp_daily["order_week"] = orders_grp_daily["order_daily"].dt.isocalendar().week

In [233]:
# 또는 한글 요일
orders_grp_daily["day_of_week"] = orders_grp_daily["order_daily"].dt.dayofweek.map({
    0: '월요일',
    1: '화요일',
    2: '수요일',
    3: '목요일',
    4: '금요일',
    5: '토요일',
    6: '일요일'
})

In [234]:
orders_grp_daily

,order_daily,담당자,email,매장명,광역,시군구,읍면동,total_order_count,total_amount,ARPU,order_week,day_of_week
0,2025-11-01,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,974500.0,121812.500000,44,토요일
1,2025-11-02,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,21,2371500.0,112928.571429,44,일요일
2,2025-11-04,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1286500.0,107208.333333,45,화요일
3,2025-11-05,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,856000.0,107000.000000,45,수요일
4,2025-11-06,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,5,518000.0,103600.000000,45,목요일
5,2025-11-07,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1148500.0,95708.333333,45,금요일
6,2025-11-08,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,1041500.0,130187.500000,45,토요일
7,2025-11-09,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,16,1654500.0,103406.250000,45,일요일
8,2025-11-11,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,4,497000.0,124250.000000,46,화요일
9,2025-11-12,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,6,665000.0,110833.333333,46,수요일


In [244]:
# 이평선
orders_grp_daily["ma_14"] = orders_grp_daily["total_amount"].rolling(window=14).mean()
orders_grp_daily["ma_28"] = orders_grp_daily["total_amount"].rolling(window=28).mean()

In [245]:
orders_grp_daily["total_amount"].pct_change(6)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6      0.068753
7     -0.302340
8     -0.613681
9     -0.223131
10     0.910232
11     0.676099
12    -0.353817
13    -0.340284
14     1.278672
15     0.972180
16     0.483578
17    -0.363896
18     1.631501
19    -0.130096
20     0.390728
21     0.255051
22    -0.892371
23    -0.472438
24    -0.197346
25    -0.184308
26     0.641587
27     0.337485
28    18.063291
29     1.082043
30     0.207527
31     0.954164
Name: total_amount, dtype: float64

In [246]:
orders_grp_daily['total_amount_WoW'] = orders_grp_daily["total_amount"].pct_change(6)

# 기준설정

| 조건                        | 점수 |
| ------------------------- | -- |
| MA14 ≥ MA28 (골든크로스)               | 0점 |
| MA14 < MA28 **(-5% 이내)** | 1점 |
| **-5% ~ -10%**            | 1점 |
| **-10% 초과 하락**            | 2점 |


In [247]:
# ma14 vs ma28
diff_rate = (orders_grp_daily["ma_14"] - orders_grp_daily["ma_28"]) / orders_grp_daily["ma_28"]

orders_grp_daily["score_trend"] = np.where(
    orders_grp_daily["ma_14"].isna() | orders_grp_daily["ma_28"].isna(),
    0,
    np.where(
        diff_rate >= 0,  # 골든크로스
        0,
        np.where(
            diff_rate >= -0.10,  # -10% 이내 (데드크로스 5~10%)
            1,
            2  # -10% 초과 하락
        )
    )
)

# 검증
orders_grp_daily[['order_daily', 'ma_14', 'ma_28', 'score_trend']].tail(10)


,order_daily,ma_14,ma_28,score_trend
22,2025-11-27,1.184286e+06,NaN,0
23,2025-11-28,1.182929e+06,NaN,0
24,2025-11-29,1.213786e+06,NaN,0
25,2025-11-30,1.131607e+06,NaN,0
26,2025-12-23,1.268214e+06,NaN,0
27,2025-12-24,1.347500e+06,1.234179e+06,0
28,2025-12-25,1.481750e+06,1.306946e+06,0
29,2025-12-26,1.484143e+06,1.270286e+06,0
30,2025-12-27,1.501893e+06,1.285643e+06,0
31,2025-12-28,1.522536e+06,1.309125e+06,0


| 조건                        | 점수 |
| ------------------------- | -- |
| total_amount ≥ ma14               | 0점 |
| total_amount < ma14 **(-10% 이내)** | 1점 |
| **-10% 초과 하락**            | 2점 |


In [248]:
# total_amount vs ma14
total_amount_diff_rate = (orders_grp_daily["total_amount"] - orders_grp_daily["ma_14"]) / orders_grp_daily["ma_14"]

orders_grp_daily["score_total"] = np.where(
    orders_grp_daily["total_amount"].isna() | orders_grp_daily["ma_14"].isna(),
    0,
    np.where(
        total_amount_diff_rate >= 0,
        0,
        np.where(
            total_amount_diff_rate >= -0.10,  # -10% 이내
            1,
            2  # -10% 초과 하락
        )
    )
)

# 검증
orders_grp_daily[['order_daily', 'total_amount', 'ma_14', 'score_total']].tail(10)


,order_daily,total_amount,ma_14,score_total
22,2025-11-27,158000.0,1.184286e+06,2
23,2025-11-28,646000.0,1.182929e+06,2
24,2025-11-29,1421500.0,1.213786e+06,0
25,2025-11-30,774500.0,1.131607e+06,2
26,2025-12-23,2585500.0,1.268214e+06,0
27,2025-12-24,2201500.0,1.347500e+06,0
28,2025-12-25,3012000.0,1.481750e+06,0
29,2025-12-26,1345000.0,1.484143e+06,1
30,2025-12-27,1716500.0,1.501893e+06,0
31,2025-12-28,1513500.0,1.522536e+06,1


In [249]:
orders_grp_daily.columns

Index(['order_daily', '담당자', 'email', '매장명', '광역', '시군구', '읍면동',
       'total_order_count', 'total_amount', 'ARPU', 'order_week',
       'day_of_week', 'ma_14', 'ma_28', 'total_amount_WoW', 'score_trend',
       'score_total', 'sum_7d', 'sum_prev_7d'],
      dtype='object')

In [251]:
# 7일 누적 vs 직전 7일 누적

# 최근 7일 누적 매출
orders_grp_daily["sum_7d"] = orders_grp_daily["total_amount"].rolling(window=7).sum()

# 직전 7일 누적 매출 (8~14일 전)
orders_grp_daily["sum_prev_7d"] = orders_grp_daily["total_amount"].rolling(window=7).sum().shift(7)


In [252]:
# 최근누적 7일 vs 직전 누적 7일
amount_7d_diff_rate = (orders_grp_daily["sum_7d"] - orders_grp_daily["sum_prev_7d"]) / orders_grp_daily["sum_prev_7d"]

orders_grp_daily["score_7d_total"] = np.where(
    orders_grp_daily["sum_7d"].isna() | orders_grp_daily["sum_prev_7d"].isna(),
    0,
    np.where(
        amount_7d_diff_rate >= 0,  # 증가 → 0점
        0,
        np.where(
            amount_7d_diff_rate >= -0.10,  # -10% 이내 하락 → 1점
            1,
            2  # -10% 초과 하락 → 2점
        )
    )
)

# 검증
orders_grp_daily[['order_daily', 'sum_7d', 'sum_prev_7d', 'score_7d_total']].tail(10)

,order_daily,sum_7d,sum_prev_7d,score_7d_total
22,2025-11-27,8792000.0,7788000.0,0
23,2025-11-28,7970000.0,8591000.0,1
24,2025-11-29,8167000.0,8826000.0,1
25,2025-11-30,7170500.0,8672000.0,2
26,2025-12-23,8806500.0,8948500.0,1
27,2025-12-24,9433000.0,9432000.0,0
28,2025-12-25,10799000.0,9945500.0,0
29,2025-12-26,11986000.0,8792000.0,0
30,2025-12-27,13056500.0,7970000.0,0
31,2025-12-28,13148500.0,8167000.0,0


In [253]:
# 최근 2주 같은 요일의 평균값
orders_grp_daily["avg_2week_same_day"] = orders_grp_daily.groupby("day_of_week")["total_amount"].transform(
    lambda x: x.rolling(window=2, min_periods=1).mean()
)

# 최근 4주 같은 요일의 평균값
orders_grp_daily["avg_4week_same_day"] = orders_grp_daily.groupby("day_of_week")["total_amount"].transform(
    lambda x: x.rolling(window=4, min_periods=1).mean()
)

orders_grp_daily[['order_daily', 'day_of_week', 'total_amount', 'avg_4week_same_day']].tail(14)

,order_daily,day_of_week,total_amount,avg_4week_same_day
18,2025-11-22,토요일,1771000.0,1115000.0
19,2025-11-23,일요일,949500.0,1516750.0
20,2025-11-25,화요일,1575000.0,1122750.0
21,2025-11-26,수요일,1646000.0,1119625.0
22,2025-11-27,목요일,158000.0,783375.0
23,2025-11-28,금요일,646000.0,1236000.0
24,2025-11-29,토요일,1421500.0,1226750.0
25,2025-11-30,일요일,774500.0,1117500.0
26,2025-12-23,화요일,2585500.0,1447500.0
27,2025-12-24,수요일,2201500.0,1456000.0


In [254]:
# 최근 2주 같은 요일의 평균값 vs 최근 4주 같은요일 평균
amount_4week_diff_rate = (orders_grp_daily["avg_2week_same_day"] - orders_grp_daily["avg_4week_same_day"]) / orders_grp_daily["avg_4week_same_day"]

orders_grp_daily["score_4week_total"] = np.where(
    orders_grp_daily["avg_2week_same_day"].isna() | orders_grp_daily["avg_4week_same_day"].isna(),
    0,
    np.where(
        amount_4week_diff_rate >= 0,
        0,
        np.where(
            amount_4week_diff_rate >= -0.10,
            1,
            2
        )
    )
)
orders_grp_daily.tail()



,order_daily,담당자,email,매장명,광역,시군구,읍면동,total_order_count,total_amount,ARPU,...,ma_28,total_amount_WoW,score_trend,score_total,sum_7d,sum_prev_7d,score_7d_total,avg_2week_same_day,avg_4week_same_day,score_4week_total
27,2025-12-24,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,15,2201500.0,146766.666667,...,1.234179e+06,0.337485,0,0,9433000.0,9432000.0,0,1923750.0,1456000.0,0
28,2025-12-25,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,20,3012000.0,150600.000000,...,1.306946e+06,18.063291,0,0,10799000.0,9945500.0,0,1585000.0,1406875.0,0
29,2025-12-26,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,10,1345000.0,134500.000000,...,1.270286e+06,1.082043,0,1,11986000.0,8792000.0,0,995500.0,1285125.0,2
30,2025-12-27,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,11,1716500.0,156045.454545,...,1.285643e+06,0.207527,0,0,13056500.0,7970000.0,0,1569000.0,1395500.0,0
31,2025-12-28,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,13,1513500.0,116423.076923,...,1.309125e+06,0.954164,0,1,13148500.0,8167000.0,0,1144000.0,1082250.0,0


In [255]:
orders_grp_daily

,order_daily,담당자,email,매장명,광역,시군구,읍면동,total_order_count,total_amount,ARPU,...,ma_28,total_amount_WoW,score_trend,score_total,sum_7d,sum_prev_7d,score_7d_total,avg_2week_same_day,avg_4week_same_day,score_4week_total
0,2025-11-01,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,974500.0,121812.500000,...,NaN,NaN,0,0,NaN,NaN,0,974500.0,9.745000e+05,0
1,2025-11-02,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,21,2371500.0,112928.571429,...,NaN,NaN,0,0,NaN,NaN,0,2371500.0,2.371500e+06,0
2,2025-11-04,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1286500.0,107208.333333,...,NaN,NaN,0,0,NaN,NaN,0,1286500.0,1.286500e+06,0
3,2025-11-05,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,856000.0,107000.000000,...,NaN,NaN,0,0,NaN,NaN,0,856000.0,8.560000e+05,0
4,2025-11-06,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,5,518000.0,103600.000000,...,NaN,NaN,0,0,NaN,NaN,0,518000.0,5.180000e+05,0
5,2025-11-07,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1148500.0,95708.333333,...,NaN,NaN,0,0,NaN,NaN,0,1148500.0,1.148500e+06,0
6,2025-11-08,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,1041500.0,130187.500000,...,NaN,0.068753,0,0,8196500.0,NaN,0,1008000.0,1.008000e+06,0
7,2025-11-09,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,16,1654500.0,103406.250000,...,NaN,-0.302340,0,0,8876500.0,NaN,0,2013000.0,2.013000e+06,0
8,2025-11-11,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,4,497000.0,124250.000000,...,NaN,-0.613681,0,0,7002000.0,NaN,0,891750.0,8.917500e+05,0
9,2025-11-12,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,6,665000.0,110833.333333,...,NaN,-0.223131,0,0,6380500.0,NaN,0,760500.0,7.605000e+05,0


In [256]:
orders_grp_daily_fin = orders_grp_daily
orders_grp_daily_fin["score"] = orders_grp_daily_fin["score_trend"] + orders_grp_daily_fin["score_total"] + orders_grp_daily_fin["score_7d_total"] + orders_grp_daily_fin["score_4week_total"]
orders_grp_daily_fin

,order_daily,담당자,email,매장명,광역,시군구,읍면동,total_order_count,total_amount,ARPU,...,total_amount_WoW,score_trend,score_total,sum_7d,sum_prev_7d,score_7d_total,avg_2week_same_day,avg_4week_same_day,score_4week_total,score
0,2025-11-01,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,974500.0,121812.500000,...,NaN,0,0,NaN,NaN,0,974500.0,9.745000e+05,0,0
1,2025-11-02,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,21,2371500.0,112928.571429,...,NaN,0,0,NaN,NaN,0,2371500.0,2.371500e+06,0,0
2,2025-11-04,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1286500.0,107208.333333,...,NaN,0,0,NaN,NaN,0,1286500.0,1.286500e+06,0,0
3,2025-11-05,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,856000.0,107000.000000,...,NaN,0,0,NaN,NaN,0,856000.0,8.560000e+05,0,0
4,2025-11-06,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,5,518000.0,103600.000000,...,NaN,0,0,NaN,NaN,0,518000.0,5.180000e+05,0,0
5,2025-11-07,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,12,1148500.0,95708.333333,...,NaN,0,0,NaN,NaN,0,1148500.0,1.148500e+06,0,0
6,2025-11-08,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,8,1041500.0,130187.500000,...,0.068753,0,0,8196500.0,NaN,0,1008000.0,1.008000e+06,0,0
7,2025-11-09,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,16,1654500.0,103406.250000,...,-0.302340,0,0,8876500.0,NaN,0,2013000.0,2.013000e+06,0,0
8,2025-11-11,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,4,497000.0,124250.000000,...,-0.613681,0,0,7002000.0,NaN,0,891750.0,8.917500e+05,0,0
9,2025-11-12,김대진 팀장,a17019@kakao.com,도리당 가락점,서울,송파구,송이로17길,6,665000.0,110833.333333,...,-0.223131,0,0,6380500.0,NaN,0,760500.0,7.605000e+05,0,0


In [260]:
# 총점 기준
# 0~2점 : 정상
# 3~4점 : 주의 (관찰)
# 5~6점 : 경고 (리뷰 대상)
# 7점 이상 : 위험 (개입)

orders_grp_daily_fin['status'] = np.where(orders_grp_daily_fin['score'] >=7, '위험',
         np.where(orders_grp_daily_fin['score'] >=5, '경고', 
                  np.where(orders_grp_daily_fin['score'] >=3, '주의', '정상')
                  )  )

# 매출 이상치 알림(임시)

## 각 스코어 기준 요약
### score_trend: MA14 vs MA28
- 골든크로스 (MA14 ≥ MA28): 0점
- -10% 이내 하락: 1점
- -10% 초과 하락: 2점

### score_total: 당일 매출 vs MA14
- 매출 ≥ MA14: 0점
- -10% 이내 하락: 1점
- -10% 초과 하락: 2점

### score_7d_total: 최근 7일 누적 vs 직전 7일 누적
- 증가: 0점
- -10% 이내 하락: 1점
- -10% 초과 하락: 2점

### score_4week_total: 최근 2주 같은 요일 매출 vs 최근 4주 같은 요일 평균
- 증가: 0점
- -10% 이내 하락: 1점
- -10% 초과 하락: 2점


### 총점 
- 0~8점 (점수가 높을수록 매출 하락 심각)


# 알림
## 알림(주의) 기준
- 총점 5점 이상이 2일 연속 발생 시 → 알림

## 즉시 경고(위험) 기준
- 총점 6점 이상 → 당일 즉시 경고
